# Chapter 8: Metaclasses and Attributes
Follow the rule of least surprise

## Item 58: Use Plain Attributes Instead of Setter and Getter Methods
Java practitioners, take note!

In [1]:
class Jimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

    def get_fios_installed(self):
        return self.fios_installed

    def set_fios_installed(self, setto: bool):
        self.fios_installed = setto

    # etc etc etc

In [2]:
jimbo = Jimmy(False)
jimbo.set_fios_installed(True)
jimbo.get_fios_installed()

True

In [3]:
# But for simple cases like this, getter/setters aren't needed:
class PythonicJimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

pimbo = PythonicJimmy(False)
pimbo.fios_installed = True
pimbo.fios_installed

True

In [4]:
# If we do need a more complicated interface for interacting with an attribute, use @property
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [5]:
jimbo2 = Jimmy(0)
for x in range(0, 9):
    jimbo2.spongebob_references = x

In [6]:
# This also lets you do type checking at runtime
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if not isinstance(references, int):
            raise ValueError("NO")
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [7]:
jimbo = Jimmy(0)
jimbo.spongebob_references = 0.5

# NOTE: you could also enforce immutability this way: raise an exception after a `hasattr` check

ValueError: NO

### Remember
- Define new class interfaces using simple public attributes and avoid defining setter and getter methods.
- Use @property to define special behavior when attributes are accessed on your objects.
- Follow the rule of least surprise and avoid odd side effects in your @property methods.
- Ensure that @property methods are fast; for slow or complex work—especially involving I/O or causing side effects—use normal methods instead.

## Item 59: Consider `@property` Instead of Refactoring Attributes

In [8]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f"Bucket(quota={self.quota})"

In [9]:
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False  # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False  # Bucket was filled, but not enough
    bucket.quota -= amount
    return True       # Bucket had enough, quota consumed

In [10]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [11]:
if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print(bucket)

Had 99 quota
Bucket(quota=1)


In [12]:
if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print(bucket)


# Problem here, is we never really know if we're blocked because we ran out of quota, or because we never had quota (during this time period)

Not enough for 3 quota
Bucket(quota=1)


In [13]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0 # we add a max_quota to track quota issued during the period
        self.quota_consumed = 0 # and this one is, well, how much we used dawg (dawg)

    def __repr__(self):
        return (
            f"NewBucket(max_quota={self.max_quota}, "
            f"quota_consumed={self.quota_consumed})"
        )

    # And NOW we use property so this new bucket has a compatible interface!
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled during the period
            self.max_quota = amount + self.quota_consumed
        else:
            # Quota being consumed during the period
            self.quota_consumed = delta

In [14]:
bucket = NewBucket(60)
print("Initial", bucket)
fill(bucket, 100)
print("Filled", bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print("Now", bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print("Still", bucket)


Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now NewBucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still NewBucket(max_quota=100, quota_consumed=99)


The magic of `@property` is it lets you make incremental progress towards a better data model.

> @property is a tool to help you address problems you’ll come across in real-world code. Don’t overuse it. When you find yourself repeatedly extending @property methods, it’s probably time to refactor your class instead of further paving over your code’s poor design.

# Item 60: Use Descriptors for Reusable `@property` Methods

Its kinda like Mixins, but for attributes.

In [15]:
# Note 2 eric: show the preamble in the book

class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._value = value

class Exam:
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [16]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print("Writing", first_exam.writing_grade)
print("Science", first_exam.science_grade)

Writing 82
Science 99


In [17]:
# All good right? NOPE!!!!
second_exam = Exam()
second_exam.writing_grade = 75
print(f"Second {second_exam.writing_grade} is right")
print(f"First  {first_exam.writing_grade} is wrong; "
      f"should be 82")


Second 75 is right
First  75 is wrong; should be 82


In [18]:
# Why this happen???
second_exam.writing_grade is first_exam.writing_grade

True

Remember long ago:
```python
def foo(bar=[]):
    ...
```

Like this older example, Python only evaluates that `Grade()` once!

In [19]:
class DictGrade:
    """
    A Grade class that keeps track of the individual values in each Exam that uses it.
    """
    def __init__(self):
        self._values = {}  # hint

    def __get__(self, instance, instance_type):
        """Return the grade value for the instance"""
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        """Set the grade value for the instance"""
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._values[instance] = value

Put on your Garbage Collector Engineer hat: what is wrong with the above implementation?

In [20]:
# Luckily Python actually has a way of making a Descriptor that can be unique to the instance that uses it:
# BEHOLD: __set_name__
class NamedGrade:
    def __set_name__(self, owner, name):
        self.internal_name = "_" + name  # creates a unique attr just for the "owning" instance

    def __get__(self, instance, instance_type):
        """Return the grade value for the instance"""
        if instance is None:
            return self
        return getattr(instance, self.internal_name)  # Note how we use getattr here,

    def __set__(self, instance, value):
        """Set the grade value for the instance"""
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        setattr(instance, self.internal_name, value)  # Note setattr

In [22]:
# Watch the magic

class NamedExam:
    math_grade = NamedGrade()
    writing_grade = NamedGrade()
    science_grade = NamedGrade()

vars(NamedExam.science_grade)

{'internal_name': '_science_grade'}

In [23]:
first_exam = NamedExam()
first_exam.math_grade = 78
first_exam.writing_grade = 89
first_exam.science_grade = 94
first_exam.__dict__

{'_math_grade': 78, '_writing_grade': 89, '_science_grade': 94}

In [26]:
# And just double checking our instance tracking works..
second_exam = NamedExam()
second_exam.writing_grade = 99
first_exam.writing_grade is second_exam.writing_grade

False

Okay, so how is it working? Take a look back to `setattr` and `getattr`. The descriptor _itself_ is not storing the data, rather, the data is now being stored on `Exam`, but with the `internal_name` that the Descriptor knows it by (for Exam instances at least).

The `writing_grade` Descriptor now knows, when an Exam class is calling `set` on it, it needs to reroute that `get` to an internal field on `Exam` called `_writing_grade`. Likewise with sets!

Now the `NamedGrade` no longer keeps a reference to any `Exam` objects, allowing full garbage collection!

### Remember
-  Reuse the behavior and validation of @property methods by defining your own descriptor classes.
- Use __set_name__ along with setattr and getattr to store the data needed by descriptors in object instance dictionaries in order to avoid memory leaks.
- Don’t get bogged down trying to understand exactly how __getattribute__ uses the descriptor protocol for getting and setting attributes.

# Item 61: Use __getattr__, __getattribute__, and __setattr__ for Lazy Attributes